## Export data from the old Database, and import processed data in the new DB

Import saved csv files using the Import/Export wizard in pdAdmin4

In [ ]:
# imports
import psycopg2
import numpy as np
import os
import pandas as pd
from dotenv import load_dotenv

In [ ]:
load_dotenv()

DB_NAME_OLD = os.getenv("DB_NAME_OLD")
DB_NAME_NEW = os.getenv("DB_NAME_NEW")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
PORT = os.getenv("PORT")
HOST = os.getenv("HOST")

# Connect to the old database
connection_old = psycopg2.connect(
    user=DB_USER,
    password=DB_PASSWORD,
    host=HOST,
    port=PORT,
    database=DB_NAME_OLD
)

# Connect to the new database
connection_new = psycopg2.connect(
    user=DB_USER,
    password=DB_PASSWORD,
    host=HOST,
    port=PORT,
    database=DB_NAME_NEW
)

### User ratings

In [ ]:
# Data from user_groa_ratings
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date, a.rating
        FROM user_groa_ratings a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_groa_ratings = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
rating = []

for row in user_groa_ratings:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2])
    date.append(row[3])
    rating.append(row[4])
    
user_groa_ratings = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'rating': rating
})
print(user_groa_ratings.shape)
user_groa_ratings.head()

(9492, 5)


id  user_id movie_id        date  rating
0  9234   312517  1051904  2016-05-23     2.0
1  7152   311074  1051904  2016-05-23     2.0
2  8853   312517  1057500  2013-02-11     2.5
3  6773   311074  1057500  2013-02-11     2.5
4  9282   312517  1059786  2017-02-16     3.0

In [ ]:
# Check for duplicates
pd.concat(g for _, g in user_groa_ratings.groupby(['id']) if len(g) > 1)

id  user_id movie_id        date  rating
787     549    42593  1895587  2019-03-15     1.5
4414    549    42593  7785302  2019-03-15     1.5
786    1054    72527  1895587  2019-03-15     1.5
4413   1054    72527  7785302  2019-03-15     1.5
785    1118    70254  1895587  2019-03-15     1.5
...     ...      ...      ...         ...     ...
8095  15624   316396  1014759  2018-12-18     3.0
757   15965   316396  1895587  2019-03-15     1.5
4384  15965   316396  7785302  2019-03-15     1.5
4559  16107   315729  1799508  2020-04-09     4.0
4605  16107   315729  2404163  2020-04-09     4.0

[118 rows x 5 columns]

In [ ]:
# drop ALL duplicate values 
user_groa_ratings.drop_duplicates(subset =['id'], keep = False, inplace = True) 

In [ ]:
# Drop the id
user_groa_ratings = user_groa_ratings.drop(['id'], axis=1)
user_groa_ratings.head()

user_id movie_id        date  rating
0   312517  1051904  2016-05-23     2.0
1   311074  1051904  2016-05-23     2.0
2   312517  1057500  2013-02-11     2.5
3   311074  1057500  2013-02-11     2.5
4   312517  1059786  2017-02-16     3.0

In [ ]:
# Add the source
user_groa_ratings['source'] = 'groa'
user_groa_ratings.head()

user_id movie_id        date  rating source
0   312517  1051904  2016-05-23     2.0   groa
1   311074  1051904  2016-05-23     2.0   groa
2   312517  1057500  2013-02-11     2.5   groa
3   311074  1057500  2013-02-11     2.5   groa
4   312517  1059786  2017-02-16     3.0   groa

In [ ]:
# Data from user_imdb_ratings
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date, a.rating
        FROM user_imdb_ratings a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_imdb_ratings = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
rating = []

for row in user_imdb_ratings:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2])
    date.append(row[3])
    rating.append(row[4])
    
user_imdb_ratings = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'rating': rating
})
print(user_imdb_ratings.shape)
user_imdb_ratings.head()

(18498, 5)


id  user_id movie_id        date  rating
0  19927       11  1396484  2017-12-31     4.5
1  20312       11  1119646  2018-12-18     4.5
2  20157       11  1119646  2018-12-18     4.5
3  19969       11  1119646  2018-12-18     4.5
4  19634       11  1119646  2018-12-18     4.5

In [ ]:
# Check for duplicates
pd.concat(g for _, g in user_imdb_ratings.groupby(['id']) if len(g) > 1)

id  user_id movie_id        date  rating
6299      57       11  0229208  2017-02-16     4.5
10702     57       11  0086879  2017-02-16     4.5
6298     291       11  0229208  2017-02-15     4.5
10701    291       11  0086879  2017-02-15     4.5
6297     832       11  0229208  2017-02-15     4.5
...      ...      ...      ...         ...     ...
10620  19533       11  0086879  2017-02-16     4.5
6216   19586       11  0229208  2017-02-16     4.5
10619  19586       11  0086879  2017-02-16     4.5
6215   19900       11  0229208  2017-02-16     4.5
10618  19900       11  0086879  2017-02-16     4.5

[170 rows x 5 columns]

In [ ]:
# drop ALL duplicate values 
user_imdb_ratings.drop_duplicates(subset =['id'], keep = False, inplace = True) 

In [ ]:
# Drop the id
user_imdb_ratings = user_imdb_ratings.drop(['id'], axis=1)

# Add the source
user_imdb_ratings['source'] = 'imdb'
user_imdb_ratings.head()

user_id movie_id        date  rating source
0       11  1396484  2017-12-31     4.5   imdb
1       11  1119646  2018-12-18     4.5   imdb
2       11  1119646  2018-12-18     4.5   imdb
3       11  1119646  2018-12-18     4.5   imdb
4       11  1119646  2018-12-18     4.5   imdb

In [ ]:
# Data from user_letterboxd_ratings
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date, a.rating
        FROM user_letterboxd_ratings a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_letterboxd_ratings = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
rating = []

for row in user_letterboxd_ratings:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2])
    date.append(row[3])
    rating.append(row[4])
    
user_letterboxd_ratings = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'rating': rating
})
print(user_letterboxd_ratings.shape)
user_letterboxd_ratings.head()

(108075, 5)


id  user_id movie_id        date  rating
0  15204    15061  0462499  2012-10-18     3.0
1  15093    15061  0114369  2012-09-21     4.0
2  15131    15061  0056869  2012-09-21     3.5
3   8000     7968  0114369  2012-09-20     4.0
4   8047     7968  0372784  2012-09-20     3.5

In [ ]:
# Check for duplicates
pd.concat(g for _, g in user_letterboxd_ratings.groupby(['id']) if len(g) > 1)

id  user_id movie_id        date  rating
81935      30        1  0086879  2020-02-13     3.5
81936      30        1  0229208  2020-02-13     3.5
80086    5023        1  0086879  2020-02-14     4.5
80093    5023        1  0229208  2020-02-14     4.5
68813    5479     5028  2651158  2013-12-14     4.0
...       ...      ...      ...         ...     ...
81843  310396   310305  0229208  2020-02-14     3.5
93743  310579    48484  0086879  2020-02-14     3.5
93744  310579    48484  0229208  2020-02-14     3.5
81661  310772   310672  0086879  2020-02-14     3.5
81664  310772   310672  0229208  2020-02-14     3.5

[1194 rows x 5 columns]

In [ ]:
# drop ALL duplicate values 
user_letterboxd_ratings.drop_duplicates(subset =['id'], keep = False, inplace = True) 

In [ ]:
# Drop the id
user_letterboxd_ratings = user_letterboxd_ratings.drop(['id'], axis=1)

# Add the source
user_letterboxd_ratings['source'] = 'letterboxd'
user_letterboxd_ratings.head()

user_id movie_id        date  rating      source
0    15061  0462499  2012-10-18     3.0  letterboxd
1    15061  0114369  2012-09-21     4.0  letterboxd
2    15061  0056869  2012-09-21     3.5  letterboxd
3     7968  0114369  2012-09-20     4.0  letterboxd
4     7968  0372784  2012-09-20     3.5  letterboxd

In [ ]:
# Shapes of ratings tables
print('shape of user_groa_ratings:', user_groa_ratings.shape)
print('shape of user_imdb_ratings:', user_imdb_ratings.shape)
print('shape of user_letterboxd_ratings:', user_letterboxd_ratings.shape)
print('Total rows:', user_groa_ratings.shape[0] + user_imdb_ratings.shape[0] + user_letterboxd_ratings.shape[0])

shape of user_groa_ratings: (9374, 5)
shape of user_imdb_ratings: (18328, 5)
shape of user_letterboxd_ratings: (106881, 5)
Total rows: 134583


In [ ]:
# Concatenate the above three dataframes
frames = [user_groa_ratings, user_imdb_ratings, user_letterboxd_ratings]
user_ratings = pd.concat(frames)
print(user_ratings.shape)
user_ratings.head()

(134583, 5)


user_id movie_id        date  rating source
0   312517  1051904  2016-05-23     2.0   groa
1   311074  1051904  2016-05-23     2.0   groa
2   312517  1057500  2013-02-11     2.5   groa
3   311074  1057500  2013-02-11     2.5   groa
4   312517  1059786  2017-02-16     3.0   groa

In [ ]:
# Save the dataframe into csv file
user_ratings.to_csv('user_ratings.csv', index=False)

### User reviews

In [ ]:
# Data from user_groa_reviews
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date, a.review, a.tags
        FROM user_groa_reviews a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_groa_reviews = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
review_title = []
review_text = []
tags = []
source = []

for row in user_groa_reviews:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2]),
    date.append(row[3])
    review_title.append(np.NaN)
    review_text.append(row[4])
    tags.append(row[5])
    source.append('groa')

user_groa_reviews = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'review_title': review_title,
    'review_text': review_text,
    'tags': tags,
    'source': source
})
print(user_groa_reviews.shape)
user_groa_reviews.head()

(315, 8)


id  user_id movie_id        date  review_title  \
0  9519   312517  1170358  2013-12-30           NaN   
1  7433   311074  1170358  2013-12-30           NaN   
2  7364   311074  1250777  2012-09-21           NaN   
3  7398   311074  1228987  2012-11-24           NaN   
4  9484   312517  1228987  2012-11-24           NaN   

                                         review_text tags source  
0  I curled my upper lip so hard that it creased....        groa  
1  I curled my upper lip so hard that it creased....        groa  
2  A marvel of filmmaking fun, fists, frisson, an...        groa  
3  Basically superfluous. The original was perfec...        groa  
4  Basically superfluous. The original was perfec...        groa

In [ ]:
# Check for duplicates
pd.concat(g for _, g in user_groa_reviews.groupby(['id']) if len(g) > 1)

id  user_id movie_id        date  review_title  \
36   7440   311074  1935179  2018-01-11           NaN   
146  7440   311074  2514592  2018-01-11           NaN   
35   9525   312517  1935179  2018-01-11           NaN   
129  9525   312517  2514592  2018-01-11           NaN   

                                           review_text tags source  
36   "The deal was for the gun, not for bullets." "...        groa  
146  "The deal was for the gun, not for bullets." "...        groa  
35   "The deal was for the gun, not for bullets." "...        groa  
129  "The deal was for the gun, not for bullets." "...        groa

In [ ]:
# drop ALL duplicate values 
user_groa_reviews.drop_duplicates(subset =['id'], keep = False, inplace = True)

# Drop the id
user_groa_reviews = user_groa_reviews.drop(['id'], axis=1)
user_groa_reviews.head()

user_id movie_id        date  review_title  \
0   312517  1170358  2013-12-30           NaN   
1   311074  1170358  2013-12-30           NaN   
2   311074  1250777  2012-09-21           NaN   
3   311074  1228987  2012-11-24           NaN   
4   312517  1228987  2012-11-24           NaN   

                                         review_text tags source  
0  I curled my upper lip so hard that it creased....        groa  
1  I curled my upper lip so hard that it creased....        groa  
2  A marvel of filmmaking fun, fists, frisson, an...        groa  
3  Basically superfluous. The original was perfec...        groa  
4  Basically superfluous. The original was perfec...        groa

In [ ]:
# Data from user_letterboxd_reviews
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date, a.review, a.tags
        FROM user_letterboxd_reviews a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_letterboxd_reviews = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
review_title = []
review_text = []
tags = []
source = []

for row in user_letterboxd_reviews:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2]),
    date.append(row[3])
    review_title.append(np.NaN)
    review_text.append(row[4])
    tags.append(row[5])
    source.append('letterboxd')

user_letterboxd_reviews = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'review_title': review_title,
    'review_text': review_text,
    'tags': tags,
    'source': source
})
print(user_letterboxd_reviews.shape)
user_letterboxd_reviews.head()

(11210, 8)


id  user_id movie_id        date  review_title  \
0    593    19614  0033870  2012-09-21           NaN   
1  11375   276525  0030287  2019-02-14           NaN   
2  12517   306631  0030287  2019-02-14           NaN   
3  12164   297984  0030287  2019-02-14           NaN   
4   8133   197098  0030287  2019-02-14           NaN   

                                         review_text tags      source  
0  Humphrey Bogart makes this film with the colde...       letterboxd  
1  "I like my convictions undiluted -- same as I ...       letterboxd  
2  "I like my convictions undiluted -- same as I ...       letterboxd  
3  "I like my convictions undiluted -- same as I ...       letterboxd  
4  "I like my convictions undiluted -- same as I ...       letterboxd

In [ ]:
# Check for duplicates
pd.concat(g for _, g in user_letterboxd_reviews.groupby(['id']) if len(g) > 1)

id  user_id movie_id        date  review_title  \
7483     91        1  2514592  2018-01-11           NaN   
8498     91        1  1935179  2018-01-11           NaN   
7537    187     5028  2514592  2018-01-11           NaN   
8552    187     5028  1935179  2018-01-11           NaN   
7536    284     5028  2514592  2018-01-11           NaN   
...     ...      ...      ...         ...           ...   
8484  12080   294333  1935179  2018-01-11           NaN   
7466  12338   301078  2514592  2018-01-11           NaN   
8481  12338   301078  1935179  2018-01-11           NaN   
7436  12634    48485  2514592  2018-01-11           NaN   
8451  12634    48485  1935179  2018-01-11           NaN   

                                            review_text  tags      source  
7483  "The deal was for the gun, not for bullets." "...  None  letterboxd  
8498  "The deal was for the gun, not for bullets." "...  None  letterboxd  
7537  "The deal was for the gun, not for bullets." "...        letterboxd  
8552  "The deal was for the gun, not for bullets." "...        letterboxd  
7536  "The deal was for the gun, not for bullets." "...        letterboxd  
...                                                 ...   ...         ...  
8484  "The deal was for the gun, not for bullets." "...        letterboxd  
7466  "The deal was for the gun, not for bullets." "...        letterboxd  
8481  "The deal was for the gun, not for bullets." "...        letterboxd  
7436  "The deal was for the gun, not for bullets." "...        letterboxd  
8451  "The deal was for the gun, not for bullets." "...        letterboxd  

[214 rows x 8 columns]

In [ ]:
# drop ALL duplicate values 
user_letterboxd_reviews.drop_duplicates(subset =['id'], keep = False, inplace = True)

# Drop the id
user_letterboxd_reviews = user_letterboxd_reviews.drop(['id'], axis=1)
user_letterboxd_reviews.head()

user_id movie_id        date  review_title  \
0    19614  0033870  2012-09-21           NaN   
1   276525  0030287  2019-02-14           NaN   
2   306631  0030287  2019-02-14           NaN   
3   297984  0030287  2019-02-14           NaN   
4   197098  0030287  2019-02-14           NaN   

                                         review_text tags      source  
0  Humphrey Bogart makes this film with the colde...       letterboxd  
1  "I like my convictions undiluted -- same as I ...       letterboxd  
2  "I like my convictions undiluted -- same as I ...       letterboxd  
3  "I like my convictions undiluted -- same as I ...       letterboxd  
4  "I like my convictions undiluted -- same as I ...       letterboxd

In [ ]:
# Shapes of user review tables
print('shape of user_groa_reviews:', user_groa_reviews.shape)
print('shape of user_letterboxd_reviews:', user_letterboxd_reviews.shape)
print('Total rows:', user_groa_reviews.shape[0] + user_letterboxd_reviews.shape[0])

shape of user_groa_reviews: (311, 7)
shape of user_letterboxd_reviews: (10996, 7)
Total rows: 11307


In [ ]:
# Concatenate the above dataframes
frames = [user_groa_reviews, user_letterboxd_reviews]
user_reviews = pd.concat(frames)
print(user_reviews.shape)
user_reviews.head()

(11307, 7)


user_id movie_id        date  review_title  \
0   312517  1170358  2013-12-30           NaN   
1   311074  1170358  2013-12-30           NaN   
2   311074  1250777  2012-09-21           NaN   
3   311074  1228987  2012-11-24           NaN   
4   312517  1228987  2012-11-24           NaN   

                                         review_text tags source  
0  I curled my upper lip so hard that it creased....        groa  
1  I curled my upper lip so hard that it creased....        groa  
2  A marvel of filmmaking fun, fists, frisson, an...        groa  
3  Basically superfluous. The original was perfec...        groa  
4  Basically superfluous. The original was perfec...        groa

In [ ]:
# Save the dataframe into csv file
user_reviews.to_csv('user_reviews.csv', index=False)

### User watched

In [ ]:
# Data from user_letterboxd_watched
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date
        FROM user_letterboxd_watched a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_letterboxd_watched = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
source = []

for row in user_letterboxd_watched:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2]),
    date.append(row[3])
    source.append('letterboxd')

user_letterboxd_watched = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'source': source
})
print(user_letterboxd_watched.shape)
user_letterboxd_watched.head()

(159231, 5)


id  user_id movie_id        date      source
0  1177      296  0371746  2012-09-20  letterboxd
1  1227      296  0033467  2012-09-20  letterboxd
2  1261      296  0114814  2012-09-20  letterboxd
3  1764      296  0443536  2012-11-05  letterboxd
4  1785      296  0358082  2012-11-09  letterboxd

In [ ]:
# Check for duplicates
pd.concat(g for _, g in user_letterboxd_watched.groupby(['id']) if len(g) > 1)

id  user_id movie_id        date      source
113951      36        1  0229208  2020-02-13  letterboxd
113952      36        1  0086879  2020-02-13  letterboxd
94814     1181      296  1655442  2012-09-20  letterboxd
94816     1181      296  1825978  2012-09-20  letterboxd
100866    1344      296  0229208  2012-09-20  letterboxd
...        ...      ...      ...         ...         ...
113693  316679   316396  2514592  2018-12-18  letterboxd
156218  316688   316396  1014759  2018-12-18  letterboxd
156219  316688   316396  2049386  2018-12-18  letterboxd
55389   317020   316396  1895587  2019-03-15  letterboxd
55393   317020   316396  7785302  2019-03-15  letterboxd

[2344 rows x 5 columns]

In [ ]:
# drop ALL duplicate values 
user_letterboxd_watched.drop_duplicates(subset =['id'], keep = False, inplace = True)

# Drop the id
user_letterboxd_watched = user_letterboxd_watched.drop(['id'], axis=1)
user_letterboxd_watched.head()

user_id movie_id        date      source
0      296  0371746  2012-09-20  letterboxd
1      296  0033467  2012-09-20  letterboxd
2      296  0114814  2012-09-20  letterboxd
3      296  0443536  2012-11-05  letterboxd
4      296  0358082  2012-11-09  letterboxd

In [ ]:
# Save the dataframe into csv file
user_letterboxd_watched.to_csv('user_watched.csv', index=False)

### User Watchlist

In [ ]:
# Data from user_letterboxd_watchlist
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date
        FROM user_letterboxd_watchlist a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_letterboxd_watchlist = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
source = []

for row in user_letterboxd_watchlist:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2]),
    date.append(row[3])
    source.append('letterboxd')

user_letterboxd_watchlist = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'source': source
})

# drop ALL duplicate values 
user_letterboxd_watchlist.drop_duplicates(subset =['id'], keep = False, inplace = True)

# Drop the id
user_letterboxd_watchlist = user_letterboxd_watchlist.drop(['id'], axis=1)
user_letterboxd_watchlist.head()

user_id movie_id        date      source
0      NaN  0114746  2012-09-20  letterboxd
1      NaN  0388795  2012-09-20  letterboxd
2      NaN  0408306  2012-09-20  letterboxd
3      NaN  0057129  2012-11-20  letterboxd
4      NaN  0114279  2012-11-20  letterboxd

In [ ]:
# How many missing values?
print(user_letterboxd_watchlist.shape)
user_letterboxd_watchlist.isnull().sum()

(92549, 4)


user_id     2086
movie_id       0
date           0
source         0
dtype: int64

In [ ]:
# Drop values with missing data
user_letterboxd_watchlist = user_letterboxd_watchlist.dropna()

# Convert user_id from float into integer
user_letterboxd_watchlist['user_id'] = user_letterboxd_watchlist['user_id'].astype(int)
user_letterboxd_watchlist.shape

(90463, 4)

In [ ]:
# Data from user_groa_watchlist
cursor_old = connection_old.cursor()
query = """SELECT a.id, a.user_id, b.movie_id, a.date
        FROM user_groa_watchlist a, imdb_movies b
        WHERE (a.name=b.primary_title AND a.year=b.start_year) OR
                (a.name=b.original_title AND a.year=b.start_year)"""
cursor_old.execute(query)
user_groa_watchlist = cursor_old.fetchall()
cursor_old.close()

id = []
user_id = []
movie_id = []
date = []
source = []

for row in user_groa_watchlist:
    id.append(row[0])
    user_id.append(row[1])
    movie_id.append(row[2])
    date.append(row[3])
    source.append('groa')

user_groa_watchlist = pd.DataFrame({
    'id': id,
    'user_id': user_id,
    'movie_id': movie_id,
    'date': date,
    'source': source
})

# drop ALL duplicate values 
user_groa_watchlist.drop_duplicates(subset =['id'], keep = False, inplace = True)

# Drop the id
user_groa_watchlist = user_groa_watchlist.drop(['id'], axis=1)
user_groa_watchlist.head()

user_id movie_id        date source
0   312517  1093842  2013-07-18   groa
1   312517  1091722  2013-07-17   groa
2   311074  1091722  2013-07-17   groa
3   312517  1069238  2012-10-23   groa
4   311074  1069238  2012-10-23   groa

In [ ]:
# Shapes of user review tables
print('shape of user_groa_watchlist:', user_groa_watchlist.shape)
print('shape of user_letterboxd_watchlist:', user_letterboxd_watchlist.shape)
print('Total rows:', user_groa_watchlist.shape[0] + user_letterboxd_watchlist.shape[0])

# Concatenate the dataframes
frames = [user_groa_watchlist, user_letterboxd_watchlist]
user_watchlist = pd.concat(frames)
print(user_watchlist.shape)
user_watchlist.head()

shape of user_groa_watchlist: (2627, 4)
shape of user_letterboxd_watchlist: (90463, 4)
Total rows: 93090
(93090, 4)


user_id movie_id        date source
0   312517  1093842  2013-07-18   groa
1   312517  1091722  2013-07-17   groa
2   311074  1091722  2013-07-17   groa
3   312517  1069238  2012-10-23   groa
4   311074  1069238  2012-10-23   groa

In [ ]:
# Save the dataframe into csv file
user_watchlist.to_csv('user_watchlist.csv', index=False)

### Movie reviews

In [ ]:
query = """SELECT movie_id FROM movies"""
cursor_new = connection_new.cursor()
cursor_new.execute(query)
movie_ids = [row[0] for row in cursor_new.fetchall()]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
movie_ids = set(movie_ids)

In [ ]:
for movie_id in movie_ids:
    print(type(movie_id))
    break

<class 'str'>


In [ ]:
# Data from user_imdb_ratings
cursor_old = connection_old.cursor()
query = """SELECT * FROM imdb_reviews"""
cursor_old.execute(query)
movie_reviews = cursor_old.fetchall()
cursor_old.close()

movie_id = []
review_date = []
user_rating = []
helpful_num = []
helpful_denom = []
user_name = []
review_text = []
review_title = []


In [ ]:

movie_reviews_lst = []

for row in movie_reviews:
    
    if row[0] not in movie_ids:
        continue
    
    movie_reviews_lst.append([row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]])

len(movie_reviews_lst)

3453257

In [ ]:
len(movie_reviews)

3462445

In [ ]:
# Save the dataframe into csv file
movie_reviews.to_csv('tab_movie_reviews.csv', sep='\t', index=False)

In [ ]:
movie_reviews = pd.read_csv("tab_movie_reviews.csv", sep="\t", dtype={"movie_id": "str"})

In [ ]:
movie_reviews.dtypes

movie_id          object
review_date       object
user_rating      float64
helpful_num        int64
helpful_denom      int64
user_name         object
review_text       object
review_title      object
dtype: object

In [ ]:
from psycopg2.extras import execute_batch

cursor_new = connection_new.cursor()
step = 1000

for ix in range(3000, len(movie_reviews), step):
    print(f"doing step: {ix}")
    
    batch = movie_reviews_lst[ix:ix+step]
    
    execute_batch(cursor_new, """
        INSERT INTO movie_reviews (movie_id, review_date, user_rating, helpful_num, helpful_denom, user_name, review_text, review_title)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """, batch
    )

doing step: 3000
doing step: 4000
doing step: 5000
doing step: 6000
doing step: 7000
doing step: 8000
doing step: 9000
doing step: 10000
doing step: 11000
doing step: 12000
doing step: 13000
doing step: 14000
doing step: 15000
doing step: 16000
doing step: 17000
doing step: 18000
doing step: 19000
doing step: 20000
doing step: 21000
doing step: 22000
doing step: 23000
doing step: 24000
doing step: 25000
doing step: 26000
doing step: 27000
doing step: 28000
doing step: 29000
doing step: 30000
doing step: 31000
doing step: 32000
doing step: 33000
doing step: 34000
doing step: 35000
doing step: 36000
doing step: 37000
doing step: 38000
doing step: 39000
doing step: 40000
doing step: 41000
doing step: 42000
doing step: 43000
doing step: 44000
doing step: 45000
doing step: 46000
doing step: 47000
doing step: 48000
doing step: 49000
doing step: 50000
doing step: 51000
doing step: 52000
doing step: 53000
doing step: 54000
doing step: 55000
doing step: 56000
doing step: 57000
doing step: 58000

In [ ]:
cursor_new.close()
connection_new.commit()

In [ ]:
batch = [[row[1][0], row[1][1], row[1][2], row[1][3], row[1][4], row[1][5], row[1][6], row[1][7]] for row in movie_reviews.iloc[0:100].iterrows()]

In [ ]:
type(batch[0][0])

int